In [1]:
import setup
setup.init_django()

In [3]:
from blog.models import BlogPost, EMBEDDING_MODEL
from decouple import config
import google.generativeai as genai

ModuleNotFoundError: No module named 'blog'

In [3]:
docs = [
    "The dog jumped over the cat",
    "The cat jumped over the dog",
    "It is very warm today",
    "The cat is yellow and the dog is red",
]

In [4]:
RECREATE_DATA = True

In [5]:
new_data = []
for i ,x in enumerate(docs):
    new_data.append(
        BlogPost(title=f"Blog Post {i + 1}", content=x, can_delete = True)
    )
    
if RECREATE_DATA:
    qs = BlogPost.objects.filter(can_delete=True)
    qs.delete()
    BlogPost.objects.bulk_create(new_data)

In [6]:
qs = BlogPost.objects.filter(can_delete=True)
qs.count()

4

In [6]:
genai.configure(
    api_key=config("GEMINI_API_KEY")
)

In [7]:
embedding_model = genai.GenerativeModel("embedding-001")

In [8]:
def get_embedding(text, model="models/embedding-001"):
    text = text.replace("\n", " ").strip()
    response = genai.embed_content(
        model=model,
        content=text,
        task_type="retrieval_document"
    )
    return response['embedding']

In [9]:
EMBEDDING_MODEL

'embedding-001'

In [11]:
query = "The dog jumped over the green cow"
query_embedding = get_embedding(query)

### Using similarity distance to annotate blogPost

In [12]:
from pgvector.django import CosineDistance
from django.db.models import F

In [14]:
for obj in qs:
    if obj.embedding is None:
        obj.embedding = get_embedding(obj.get_embedding_text_raw())
        obj.save()
        
qs = BlogPost.objects.filter(can_delete=True)

In [18]:
qs = BlogPost.objects.annotate(
    distance   = CosineDistance('embedding', query_embedding),
    similarity = F("distance")                             
).order_by("-distance")[:4]

for obj in qs:
    print(obj.title, obj.distance, obj.similarity * 100)

Blog Post 4 0.20695340840619914 20.695340840619913
Blog Post 4 0.20695340840619914 20.695340840619913
Blog Post 3 0.20682441260568352 20.682441260568353
Blog Post 3 0.20682441260568352 20.682441260568353


### Connecting to a Pooled DB

In [6]:
from decouple import config